In [1]:
%matplotlib
import cma

Using matplotlib backend: MacOSX


In [13]:
import numpy as np
import cma 
def flat_warning_preventer():
    return 1.01e-2 + 1e-2 * np.exp(np.random.randn())
def bin_val(x):
    return np.dot(2.**np.arange(len(x)), x) + 0 * flat_warning_preventer()
def leadingones(x):
    for i in range(len(x)):
        if x[i] == 0:
            break
    else:
        i = len(x)
    return len(x) - i + 0 * flat_warning_preventer()
def onemax(x):
    return len(x) - sum(x) + 0 * flat_warning_preventer()
def frand(x):
    return np.exp(np.random.randn())

In [19]:
import cma

class BinaryCMAEvolutionStrategy(cma.CMAEvolutionStrategy):
    @staticmethod
    def binarise(X):
        """transform output of `ask` as input to binary domain objective
        """
        return [1.0 * (x > 0) for x in X]
    def __init__(self, *args, **kwargs):
        cma.CMAEvolutionStrategy.__init__(self, *args, **kwargs)
        self.logger.name_prefix += 'bin'
    def ask_bin(self, *args, **kwargs):
        """return binarised and original solutions, 
        
        to be used to evaluate the objective and to call `tell`,
        respectively.
        """
        X = self.ask(*args, **kwargs)
        return self.binarise(X), X
    def tell(self, *args, **kwargs):
        res = cma.CMAEvolutionStrategy.tell(self, *args, **kwargs)
        self._finalise_iter()
        return res
    def _finalise_iter(self):
        self.sigma = 1
        # seems never necessary but shouldn't hurt either if cmean is small enough:
        self.mean -= (self.opts['CMA_cmean'] / 1e3) * np.sign(self.mean)

class DiscretisedPopulation(list):
    """a list (population) of integer vectors
    """
    def __init__(self, X):
        """`X` is an array of solution vectors"""
        super().__init__(np.array(x, dtype=int) for x in X)
        # self._sampled_pop = X

class DiscreteCMAEvolutionStrategy(cma.CMAEvolutionStrategy):
    def __init__(self, x0, sigma, arities, *args):
        N = len(x0)
        min_arity = np.min(arities)
        opts = {
            'CMA_cmean': 'min((1, %f * (popsize / (10 + popsize/N))))' % (arities / (N + arities)),
            'CMA_on': (min_arity + 1) / (min_arity + 30),
            'bounds': [0, arities - 1e-11],
            'maxstd': 1 * (arities - 1) / 4,  # maxstd < bounds/2
            # maxstd=1/2 costs some performance on binval 80D
            'minstd': max((1, N / arity)) / 2 / (N + 1),
            }
        if args:
            for key in opts:
                if key in args[0]:
                    print("""Option '%s'=%s will be overwritten to 
                    %s. This may seriously hamper DiscreteCMA.""" 
                          % (key, opts[key], args[0][key]))
            opts.update(args[0])
            args[0].update(opts)
        else:
            args = (opts,)
        print(args[0])
        super().__init__(x0, sigma, *args)
        self.logger.name_prefix += 'bin'
        self._arity_pop = {'keys':[], 'values':[]}
    def ask(self, *args, **kwargs):
        """return discretised solutions as `DiscretisedPopulation` instance.
        
        To be used to evaluate the objective and to call `tell`,
        respectively.
        
        Caveat: resampling single solutions will fail.
        
        Details: `tell` uses the hidden attribute `_sampled_pop`.

        TODO: this is not a sustainable solution for a mixed-cma
        version, as it defeats separation of the return value of
        `ask` and the input arg to `tell`. Archiving all current
        genotypic solutions (as already done?) is the best approach
        I can think of right now.
        """
        X = super().ask(*args, **kwargs)
        Xdis = DiscretisedPopulation(X)
        self._arity_pop['keys'].extend(Xdis)
        self._arity_pop['values'].extend(X)
        return Xdis
    ask.__doc__ += cma.CMAEvolutionStrategy.ask.__doc__

    def tell(self, X, fvals, *args, **kwargs):
        # TODO: find X in 'keys' first
        X = self._arity_pop['values']
        if any(np.asarray(fvals) < 1e-2):
            fvals = [fval + 1 * flat_warning_preventer() for fval in fvals]
        res = super().tell(X, fvals, *args, **kwargs)
        self._finalise_iter()
        self._arity_pop = {'keys':[], 'values':[]}
        return res
    def _finalise_iter(self):
        """does nothing for the time being"""
        return
        # seems never necessary but shouldn't hurt either if cmean is small enough:
        self.mean -= (self.opts['CMA_cmean'] / 1e3) * np.sign(self.mean)

class MixedBinarySphere:
    """the binary variables define the optimum for the continuous"""
    def __init__(self, function, arity=2, xopt=None):
        """function is the continuous function, xopt the binary opt"""
        self.function = function
        self.arity = arity
        self._xopt = xopt
    def xopt(self, dimension):
        if self._xopt is not None:
            return self._xopt
        return np.arange(dimension) % self.arity
    def __call__(self, x_bin, x_cont=None):
        x_bin, x_cont = map(np.asarray, (x_bin, x_cont))
        if not np.all(x_bin == np.asarray(x_bin, int)):
            raise ValueError("variables are not integer " +
                             str(x_bin))
        f = 0
        if x_cont.shape:  # hack for is not None
            # x_opt = x_bin[:len(x_cont)]
            i_opt = np.arange(len(x_cont), dtype=int) % len(x_bin)
            f = self.function((x_cont - x_bin[i_opt]))
        # TODO: factor should be exposed
        return f + 1e0 * sum((x_bin - self.xopt(len(x_bin)))**2) 



In [20]:
def smart_sortings(binary_X, f_vals):
    """return smart sortings conditioned to same binary values"""
    X_dict = {tuple(x): [] for x in binary_X}
    len_ = len(X_dict)
    if len(f_vals) == len_:
        return f_vals, f_vals
    for x, f in zip(binary_X, f_vals):
        X_dict[tuple(x)] += [f]
    assert len(X_dict) == len_
    raise NotImplementedError(X_dict)

# TODO: better use same weights for same f-value
# TODO: check again binary values for tell
dim_bin = 40
dim = 10
arity = 2
function = MixedBinarySphere(cma.ff.sphere, arity, None)
function = leadingones
if 1 < 3:
    es_bin = DiscreteCMAEvolutionStrategy(dim_bin * [1], arity / 5,
         arity,
         {#'popsize': '3*N', 
          #'CMA_cmean': 0.1,  # es.popsize, 
          # 'CMA_on': 0.51, 
          # 'CMA_diagonal': True,
          'maxiter': 30 * dim_bin,
          
         })
    while not es_bin.stop() and not all(es_bin.mean > 1.4) and not all(es_bin.mean < 0.6):
        X_bin = es_bin.ask()
        fvals = [function(xx) for xx in X_bin]
        es_bin.tell(X_bin, fvals)
        es_bin.logger.add()
        es_bin.disp()
    es_bin.result_pretty()

elif 1 < 3:
    es = cma.CMAEvolutionStrategy(dim * [1], 1, {
        'maxiter': 1500, #'popsize': 50
    })
    es_bin = DiscreteCMAEvolutionStrategy(dim_bin * [1], 0.25,
         arity,
         {'popsize':es.popsize})
    while not es.stop():
        X_bin = es_bin.ask()
        X = es.ask()
        fvals = [function(*xx) for xx in zip(X_bin, X)]
        # smart_sortings(X_bin, fvals)
        es_bin.tell(X_bin, fvals)
        es.tell(X, fvals)
        es.logger.add()
        es_bin.logger.add()
        es.disp()
    es.result_pretty()
    cma.plot();
cma.plot('outcmaesbin', plot_mean=1);

{'maxiter': 1200, 'CMA_cmean': 'min((1, 0.047619 * (popsize / (10 + popsize/N))))', 'CMA_on': 0.09375, 'bounds': [0, 1.99999999999], 'maxstd': 0.25, 'minstd': 0.24390243902439024}
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 40 (seed=720523, Thu Dec  7 21:04:29 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 3.700000000000000e+01 1.0e+00 2.50e-01  2e-01  3e-01 0:00.0
    2     30 3.700000000000000e+01 1.0e+00 2.44e-01  2e-01  2e-01 0:00.0
    3     45 3.600000000000000e+01 1.0e+00 2.44e-01  2e-01  2e-01 0:00.0
  100   1500 2.900000000000000e+01 1.1e+00 2.49e-01  2e-01  3e-01 0:00.3
  200   3000 2.200000000000000e+01 1.3e+00 2.48e-01  2e-01  3e-01 0:00.5
  300   4500 1.700000000000000e+01 1.3e+00 2.46e-01  2e-01  3e-01 0:00.9
  400   6000 1.100000000000000e+01 1.4e+00 2.46e-01  2e-01  3e-01 0:01.2
  500   7500 4.000000000000000e+00 1.4e+00 2.45e-01  2e-01  3e-01 0:01.4
  600   9000 1.000000000000000e+00 1.4e+00 2.50e-01  2e-01  3e-01 0:01.7

In [5]:
es_bin.opts['CMA_cmean']

0.43955999999999995

In [6]:
# cma.plot('outcmaesbin')
# %matplotlib
# %pylab
# figure(1);

Using matplotlib backend: MacOSX


In [7]:
es_bin.logger.load()

In [8]:
figure(1); gcf().clear()
bh = es_bin.boundary_handler
m = es_bin.logger.data['xmean']
plot(m[:,0], [bh.inverse(bh.transform(mm)) for mm in m[:, 5:]]);


NameError: name 'figure' is not defined

In [ ]:
show()